# Deep Learning with SkyDL
**Setup software libraries**

In [1]:
from SkyDL import skydl
import env

**Create `SkyDL` object**

In [2]:
SkyTrainer = skydl.Trainer(privatekey_path = env.privatekey_path)

**Get token**

In [3]:
SkyTrainer.get_token(email='iker.sanchez@vizzuality.com')

Skydipper login password: ·········


## Database

**Retrive table names of the database**

In [4]:
SkyTrainer.table_names

['model', 'image', 'model_versions', 'dataset']

**Retrieve table from database**

In [5]:
SkyTrainer.get_table(table_name='dataset')

,slug,name,bands,rgb_bands,provider
0,Sentinel-2-Top-of-Atmosphere-Reflectance,Sentinel 2 Top-of-Atmosphere Reflectance,"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8...","['B4', 'B3', 'B2']",gee
1,Landsat-7-Surface-Reflectance,Landsat 7 Surface Reflectance,"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'nd...","['B3', 'B2', 'B1']",gee
2,Landsat-8-Surface-Reflectance,Landsat 8 Surface Reflectance,"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B1...","['B4', 'B3', 'B2']",gee
3,USDA-NASS-Cropland-Data-Layers,USDA NASS Cropland Data Layers,"['landcover', 'cropland', 'land', 'water', 'ur...",['landcover'],gee
4,USGS-National-Land-Cover-Database,USGS National Land Cover Database,['impervious'],['impervious'],gee
5,Lake-Water-Quality-100m,Lake Water Quality 100m,['turbidity_blended_mean'],['turbidity_blended_mean'],gee


## Skydipper datasets for Deep Learning

In [6]:
#SkyTrainer.datasets_api 

## Image composites

In [7]:
SkyTrainer.composite(slugs=['Sentinel-2-Top-of-Atmosphere-Reflectance', 'Lake-Water-Quality-100m'],\
              init_date = '2019-01-21', end_date = '2019-01-31', zoom=6)

Check the composites as ee.Images

In [8]:
SkyTrainer.composites

[<ee.image.Image at 0x109e582e8>, <ee.image.Image at 0x11812b630>]

## Creation of Geostore

We select the areas from which we will export the training data.

In [9]:
train_atts = {"type":"FeatureCollection",
              "features":[
                  {"type":"Feature",
                   "properties":{},
                   "geometry":{
                       "type":"Polygon",
                       "coordinates":[[[-0.406494140625,38.64476310916202],
                                       [0.27740478515625,38.64476310916202],
                                       [0.27740478515625,39.74521015328692],
                                       [-0.406494140625,39.74521015328692],
                                       [-0.406494140625,38.64476310916202]]]
                   }
                  },
                  {"type":"Feature",
                   "properties":{},
                   "geometry":{
                       "type":"Polygon",
                       "coordinates":[[[-1.70013427734375,35.15135442846945],
                                       [-0.703125,35.15135442846945],
                                       [-0.703125,35.94688293218141],
                                       [-1.70013427734375,35.94688293218141],
                                       [-1.70013427734375,35.15135442846945]]]
                   }
                  }
              ]
             }

train_atts = {"type":"FeatureCollection",
              "features":[
                  {"type":"Feature",
                   "properties":{},
                   "geometry":{
                       "type":"Polygon",
                       "coordinates":[[[-0.406494140625,38.64476310916202],
                                       [0.28,38.64476310916202],
                                       [0.28,39.74521015328692],
                                       [-0.406494140625,39.74521015328692],
                                       [-0.406494140625,38.64476310916202]]]
                   }
                  },
                  {"type":"Feature",
                   "properties":{},
                   "geometry":{
                       "type":"Polygon",
                       "coordinates":[[[-1.70013427734375,35.15135442846945],
                                       [-0.703125,35.15135442846945],
                                       [-0.703125,35.94688293218141],
                                       [-1.70013427734375,35.94688293218141],
                                       [-1.70013427734375,35.15135442846945]]]
                   }
                  }
              ]
             }

valid_atts = None
test_atts = None

In [10]:
SkyTrainer.create_geostore_from_geojson(attributes=[train_atts, valid_atts, test_atts])

Number of training polygons: 2


Check geostore object

In [11]:
SkyTrainer.geostore

{'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {'name': 'training'},
    'geometry': {'type': 'MultiPolygon',
     'coordinates': [[[[-0.406494140625, 38.64476310916202],
        [0.28, 38.64476310916202],
        [0.28, 39.74521015328692],
        [-0.406494140625, 39.74521015328692],
        [-0.406494140625, 38.64476310916202]]],
      [[[-1.70013427734375, 35.15135442846945],
        [-0.703125, 35.15135442846945],
        [-0.703125, 35.94688293218141],
        [-1.70013427734375, 35.94688293218141],
        [-1.70013427734375, 35.15135442846945]]]]}}]}}

In [12]:
SkyTrainer.nPolygons

{'training': 2}

Check geostore object on a server and display it on map

In [13]:
#SkyTrainer.multipolygon

In [14]:
#SkyTrainer.multipolygon.map()

## Data pre-processing

We normalize the composite images to have values from 0 to 1.

In [15]:
SkyTrainer.normalize_images(scale=100, norm_type='geostore')

Check the normalized composites as ee.Images

In [16]:
SkyTrainer.norm_composites

[<ee.image.Image at 0x1181d0898>, <ee.image.Image at 0x1181bf2e8>]

### Select input/output bands

In [17]:
SkyTrainer.select_bands(input_bands = ['B2','B3','B4','B5','ndvi','ndwi'],\
                 output_bands = ['turbidity_blended_mean'])

In [18]:
SkyTrainer.images

,dataset_id,bands_selections,scale,init_date,end_date,bands_min_max,norm_type
0,0,"['B2', 'B3', 'B4', 'B5', 'ndvi', 'ndwi']",100.0,2019-01-21,2019-01-31,"{""B11_max"": 10857.5, ""B11_min"": 7.0, ""B12_max""...",geostore
1,5,['turbidity_blended_mean'],100.0,2019-01-21,2019-01-31,{},geostore
2,0,"['B2', 'B3', 'B4', 'B5', 'ndvi', 'ndwi']",100.0,2019-01-21,2019-01-31,"{""B11_max"": 10857.5, ""B11_min"": 7.0, ""B12_max""...",geostore


## Create TFRecords for training

In [19]:
SkyTrainer.export_TFRecords(sample_size = 20000, kernel_size = 1)

In [20]:
SkyTrainer.versions

,model_id,model_architecture,input_image_id,output_image_id,geostore_id,kernel_size,sample_size,training_params,version,data_status,training_status,eeified,deployed
0,-9999,,0,1,cc1d5d3dee2c8a6e9a4d10f74f7747a9,1,20000,{},-9999,COMPLETED,,False,False


## Training the model in AI Platform

In [21]:
SkyTrainer.train_model_ai_platform(model_type='MLP', model_output='regression', model_architecture='sequential1',\
                                   model_name='water_quality', batch_size=4)

Model name assigned: water_quality with description: 
Create new version
Creating training job: job_v1581696278
{'jobId': 'job_v1581696278', 'trainingInput': {'scaleTier': 'CUSTOM', 'masterType': 'large_model_v100', 'packageUris': ['gs://geo-ai/Train/trainer-0.2.tar.gz'], 'pythonModule': 'trainer.task', 'args': ['--params-file', 'Models/0/1581696278/training_params.json'], 'region': 'us-central1', 'runtimeVersion': '1.14', 'jobDir': 'gs://geo-ai/Models/0/1581696278/', 'pythonVersion': '3.5'}, 'createTime': '2020-02-14T16:04:43Z', 'state': 'QUEUED', 'trainingOutput': {}, 'etag': 'H06ODDFk920='}


TypeError: Got an unexpected keyword argument "body"

In [22]:
SkyTrainer.training_params

{'bucket': 'geo-ai',
 'base_names': ['training_pixels', 'validation_pixels', 'test_pixels'],
 'data_dir': 'gs://geo-ai/Data/0_1/cc1d5d3dee2c8a6e9a4d10f74f7747a9/1/20000',
 'in_bands': ['B2', 'B3', 'B4', 'B5', 'ndvi', 'ndwi'],
 'out_bands': ['turbidity_blended_mean'],
 'kernel_size': 1,
 'training_size': 7071,
 'validation_size': 2309,
 'model_type': 'MLP',
 'model_output': 'regression',
 'model_architecture': 'sequential1',
 'output_activation': '',
 'batch_size': 4,
 'epochs': 25,
 'shuffle_size': 2000,
 'learning_rate': 0.001,
 'loss': 'mse',
 'metrics': ['mse'],
 'job_dir': 'gs://geo-ai/Models/0/1581696278/'}

In [23]:
SkyTrainer.versions

,model_id,model_architecture,input_image_id,output_image_id,geostore_id,kernel_size,sample_size,training_params,version,data_status,training_status,eeified,deployed
0,0,sequential1,0,1,cc1d5d3dee2c8a6e9a4d10f74f7747a9,1,20000,"{""bucket"": ""geo-ai"", ""base_names"": [""training_...",1581696278,COMPLETED,,False,False


In [24]:
SkyTrainer.models

,model_name,model_type,model_output,model_description,output_image_id
0,water_quality,MLP,regression,,1


In [25]:
SkyTrainer.versions.drop(index=0)

,model_id,model_architecture,input_image_id,output_image_id,geostore_id,kernel_size,sample_size,training_params,version,data_status,training_status,eeified,deployed
